# Simplify

The main functionality of PyZX is the ability to optimize quantum circuits. The main optimization methods work by converting a circuit into a ZX-diagram, simplifying this diagram, and then converting it back into a quantum circuit. This process is explained in the next section. There are also some basic optimization methods that work directly on the quantum circuit representation. This is detailed in the section [Gate-level optimization](#gate-level-optimization) .


## Optimizing circuits using the ZX-calculus

PyZX allows the simplification of quantum circuits via a translation to the ZX-calculus. To demonstrate this functionality, let us generate a random circuit using :func:`~pyzx.generate.CNOT_HAD_PHASE_circuit`::



	c = zx.generate.CNOT_HAD_PHASE_circuit(qubits=8, depth=100)
	print(c.stats())

To use the ZX-diagram simplification routines, the circuit must first be converted to a ZX-diagram::

	g = c.to_graph()

We can now use any of the built-in simplification strategies for ZX-diagrams. The most powerful of these is :func:`~pyzx.simplify.full_reduce`::

	zx.full_reduce(g, quiet=False) # simplifies the Graph in-place, and show the rewrite steps taken.
	g.normalize() # Makes the graph more suitable for displaying
	zx.draw(g) # Display the resulting diagram

This rewrite strategy implements a variant of the algorithm described in `this paper <https://arxiv.org/abs/1903.10477>`__.
The resulting diagram most-likely does not resemble the structure of a circuit. In order to extract an equivalent circuit from the diagram, we use the function :func:`~pyzx.extract.extract_circuit`.

Simply use it like so::

	c_opt = zx.extract_circuit(g.copy())

For some circuits, :func:`~pyzx.extract.extract_circuit` can result in quite large circuits involving many CNOT gates. If one is only interested in optimizing the T-count of a circuit, the extraction stage can be skipped by using the *phase-teleportation* method of `this paper <https://arxiv.org/abs/1903.10477>`__. This applies ``full_reduce`` in such a way that only phases are moved around the circuit, and all other structure remains intact::

	g = c.to_graph()
	zx.teleport_reduce(g)
	c_opt = zx.Circuit.from_graph(g) # This function is able to reconstruct a Circuit from a Graph that looks sufficiently like a Circuit


## Gate-level optimization